## RSA - The Project

Name:  Mitch Henderson

Moodle Email: mihe2568@colorado.edu

Did you name your file "cuidentikey".ipynb ? Answer yes

<hr />

# Table of Contents

### 1. Introduction

### 2. RSA Code Package
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.1 Basic tool set
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.2 First tool set
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.3 2nd tool set

### 3. RSA More Code
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3.1 Step 1: Get Keys
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3.2 Step 2: Encode
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 3.3 Step 3: Decode

### 4. How the RSA demo works - This will be a mix of code and text blocks 
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 4.1 Get Keys
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 4.2 Encode
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 4.3 Decode

### 5. Exchanging keys and codes

### 6. Brute force method of finding factors of n

### 7. Breaking codes without a private key

### 8. How many codes did I break?

### 9. Improving the brute force method of factoring

### 10. Conclusion

### 1. Introduction to your RSA Package and Project.  Give an overview of your project and what you have learned.

* Put introduction/overview here. This should be something someone could read who has never seen your project to understanding what is coming. 

### Here you are going to see an implementation of the security protocol comonly referred to as RSA.  I start by converting text into ascii characters.  From there I take these characters and expand them into their binary expansion.  We create two tool sets.  

### The first tool set will run an algorithm called Fast Modular Exponentiation. This will be used to first encrypt the message, and then subsequently to decrypt the message. It will be sped up using the binary expansion of the ascii characters to help decrypt other people's codes.

### Then, also in the first toolset, I run another algorithm called Exapnded Euclidean Algorithm. This will be used to determine the value of "d". "d" is the inverse of e modulo (p-1)(q-1).  We keep these values d and n as our private key.

### The functions in the second toolset will generate the public and private key pairs which will then be used to create a ciphertext using the public key and then decode the same using the private key. So here in my second toolset, find_public_key, I have a function that will produce two values, e and n. This function will run a loop to find e such that e is relatively prime to (p - 1) (q - 1) and not equal to p or q.

### Then, in our second toolset, find_private_key, we have a function to find the decryption exponent d such that d is the modular inverse of e.  This uses the Extended Eucliean Algorithm and returns d: the decryption component. 

### Then, in my third part, I use the Encode function Convert_Text from Second tool set and get a list of numbers. I encode each of those numbers using n and e and return the encoded cipher_text. Using the public key, the Encode function will encode a message and generate the corresponding cipher_text.

### Here, in my third part, we use the Decode function.  the cipher_text will be a list of integers. First, I decrypt each of those integers using n and d. I use the function that converts the integers to a string defined in second toolset to recover the original message as a string. Using the private key, the Decode function will decode a ciper_text and recover the original message.



### 2.1
#### Basic tool set

These are functions that you'll need to pre-process the messages before the messages are encoded and decoded by the RSA algorithm. That is the reason we will be defining them first.



In [1]:
def Convert_Text(_string):
    """
    Define this function such that it takes in a simple 
    string such as "hello" and outputs the corresponding
    standard list of integers (ascii) for each letter in the word hello.
    For example:
    _string = hello
    integer_list = [104, 101, 108, 108, 111]
    """
    integer_list = [] #setting the integer list
    for letter in _string:
        result = ord(letter)#the result is the value of the byte when the argument is an 8-bit string
        integer_list.append(result) #appending to list
    
    return integer_list

In [2]:
def Convert_Num(_list):
    """
    Do the opposite of what you did in the Convert_Text
    function defined above.
    
    Define this function such that it takes in a simple 
    string such as "hello" and outputs the corresponding
    list of integers for each letter in the word hello.
    For example:
    _list = [104, 101, 108, 108, 111]
    _string = hello
    """
    _string = ''
    for i in _list: #iterating through i in _list
        _string += chr(i) #returns the string representing the character whose Unicode code point is an integer.
    return _string

In [3]:
# 234
# 0 - 0
# 1 - 1
# 2 = 10
# 3 = 11
# 4 = 100

# 234 in base 10 is what in binary?
# 234-128 = 106-64 = 42-32 = 10-8 = 2-2 = 0

#   1  1  1  0  1  0  1  0
# 128 64 32 16  8  4  2  1


# 234 =  11101010
# 0*1
# 1*2
# 0*4
# 1*8
# 0*16
# 1*32
# 1*64
# 1*128






In [4]:
def Convert_Binary_String(_int):
    """
    Here, you need to define a function that converts an integer to
    a string of it's binary expansion.
    
    For example:
    _int = 345
    bits = 101011001
    """
    bits = ''
    value = 1
    # keep doubling value until it exceeds int: 1 2 4 8 16 32 64 128 256 512 (for 345 eg)
    while value <= _int:
        value = value * 2
    # divide value in half so it's just barely smaller (or possibly equal)
    value = value // 2 # (256, eg for 345)
    while value > 0: # do this until value gets back to 1
        if _int >= value: # if we can take "value" out of int
            _int = _int - value
            bits = bits + "1" # add a 1
        else:
            # int stays the same, but add a 0 to indicate the inability to do so
            bits = bits + "0"
        value = value//2 # divide value by 2, to progress through digits (256 128 64 etc etc)
    
    return bits

Convert_Binary_String(345)

'101011001'

Now that you're done with the basic toolset we'll move on to the first tool set which are actually involved in the RSA system.

### 2.2 
#### First tool set.



In [5]:
def FME( b, n, m):
    """
    1. Using the fast modular exponentiation algorithm.
    the below function should return b**n mod m.
    As described on page 254. (however, you may input the exponent and 
    then convert it to a string - the book verison imports the binary expansion)  
    2. You should use the function defined above
    Convert_Binary_String()
    3. Use this string to test which components are used in the calculation.
    4. Yes, there are many other ways to do this. 
***UPDATE*** You may use the function you developed in Week 6 - but be sure it is your own work commented etc.. and change inputs as needed.
    """  
    
    n_bin = Convert_Binary_String(n)
    r = 1 #starting with r equal to 1
    for bit in n_bin: #looping through this for loop
        r = r*r #start with r equal to r times r
        if bit == "1": #if bit is equal to 1 then
            r = (r * b) #setting r = to r * b
        else:
            pass
        #print(r, m)
        r = r%m # here I set r equal to r modulo m
    '''
    eprime = 0
    if m == 1:
        return 0
    result = 1
    while eprime < exponent:
        result = (result * b) % m
        eprime = eprime + 1
    return result
    '''
    return r #finally returning r
    
# note: input throws off the notebook
#b = int(input("Enter base: "))
#n = int(input("Enter exponent: "))
#m = int(input("Enter modulus: "))
#print("Answer is ", FME(b, n, m) )
FME(100, 200, 50)


0

In [6]:
def Euclidean_Alg(a, b):
    """
    1. Calculate the Greatest Common Divisor of a and b.
    
    2. Unless b==0, the result will have the same sign as b (so that when
    b is divided by it, the result comes out positive).
    The function must return a single integer 'x' which is
    the gcd of a and b.
    """
    while  b != 0: #as long as b is not zero
        
        remainder = a % b # get remainder
        
        a = b #setting a equal to b
        b = remainder #setting b to the remainder
    x = a #setting x now equal to a
    return x 

result = Euclidean_Alg(34,51)
print(result)



17


### 2.3
#### Second tool set

Here we will implement the meat of the RSA cryptosystem. The functions below will generate the public and private key pairs which will then be used to create a ciphertext using the public key and then decode the same using the pirvate key.



In [7]:
def get_primes_to_sqrt(n):
    # we know that the prime we find must be less than or equal to the square root of n
    # [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    # [F  F  T  T  T  T  T  T  T  T   T   T   T] 
    cap = int((n+1)**0.5)+1 # set the cap to the square root of n+1 (plus an extra 1 for wiggle room)
    # primes variable will represent whether the number at that index is prime.  eg: primes[7] == True, primes[8] == False
    primes = [True]*cap # create that many trues (so if number is 10,000, cap will be 101, and 101*101 >= 10,000)
    primes[0] = False # 0 can't be prime
    primes[1] = False # 1 also can't be prime
    p = 1 # so starting from 1
    
    while p < cap: # as long as p is less than the cap
        p = p + 1 # add 1 to it
        while p < cap and primes[p] == False: # and as long as primes[p] has already been determiend as not prime
            p = p + 1 # add 1 until it hits a number that IS prime (since it's not divisible by any smaller numbers)
        for i in range(p*p, cap, p): # now starting from p, make every p'th number non-prime (not counting p)
            primes[i] = False 
            # 0f 1f 2T 3T 4f 5T 6f 7T 8f 9f 10f 11T 12f 13 14f 15f 16f 17 18f 19 20f 21f 22f 23..
            # note; I START AT p*p because EVERYTHING between 11 and 121 that's divisble by 11 would have to be divisible by a number < 11
            # eg: 77 looks like it might be prime, but it's divisble by 11, and ALSO by 7 (which is less than 11) so no need to check 77.
    prime_numbers = [] # prepare the prime number box
    for i in range(0, cap): # go through the entire collection of trues and falses
        if primes[i] == True: # and if a value is true
            prime_numbers.append(i) # that index (i) is prime
    return prime_numbers # return the numbers

def Find_Public_Key_e(p, q):
    """
    Implement this function such that
    it takes 2 primes p and q.
    
    Use the gcd function that you have 
    defined before.
    
    The function should return 2 elements as follows:
    public key: n
    public key: e
    
    
    HINT: this function will run a loop to find e such 
    that e is relatively prime to (p - 1) (q - 1) 
    and not equal to p or q.
    """
    n = p*q #from RSA algorithm
    m = (p-1)*(q-1) #from the RSA algorithm
    
    primes = get_primes_to_sqrt(n) #setting primes to get_primes_to_square root
    for e in primes: 
        if Euclidean_Alg(m, e) == 1:  #if gcd is equal to 1
            return (e, n) #then return e and n
        
    return (e, n)

Find_Public_Key_e(29, 31)

(11, 899)

In [8]:
def egcd(a, b):
    #https://crypto.stackexchange.com/questions/19444/rsa-given-q-p-and-e
    #I used this because my earlier posted method was simply taking too long. I needed to see inspiration from where this could work to find d.
    x = 0
    y = 1
    u = 1
    v = 0
    while a != 0:
        q = b//a # q is b/a rounded down
        r = b%a # r is the remainder of b/a
        m = x-u*q # m is x minus u*q
        n = y-v*q # n is y minus v*q
        b = a # move a to b, r to a, u to x, v to y, m to u , n to v, and b to gcd
        a = r # then repeat until a becomes 0
        x = u
        y = v
        u = m
        v = n
        gcd = b
    return x

def Find_Private_Key_d(e, p, q):
    """
    Implement this method
    to find the decryption exponent d such that
    d is the modular inverse of e. 
    
    This will use the Extended Eucliean Algorithm
    
    This function should return the following:
    d: the decryption component.
    """
    m = (p-1)*(q-1) #from the RSA algorithm
    d = egcd(e, m) #setting do to the gcd of e and m
    # cleanup for when it is negative
    while d <= 0:
        d += m #adding until we get to a positive d
    d
    
    return d, p*q

e = 113
p = 160481219
q = 179424673
d, n = Find_Private_Key_d(e, p, q)
d, n

(23697955405914833, 28794290241716387)

### 3.
#### Putting things all together.

1. In this part, you will define two functions `Encode` and `Decode` which will use the public and private keys that you calculated using the above 2 functions in the third toolset.
2. Using the public key, the `Encode` function will encode a message and generate the corresponding cipher_text.
3. Using the private key, the `Decode` function will decode a ciper_text and recover the original message.



In [9]:
def Encode(n, e, message):
    """
    Here, the message will be a string of characters.
    Use the function Convert_Text from 
    Second tool set and get a list of numbers.
    
    Encode each of those numbers using n and e and
    return the encoded cipher_text.
    """
    cipher_text = [] #setting up the cipher_text
    for letter in message: #creating the for loop to encoded each character
        ascii_val = ord(letter) # returning the integer representing the Unicode code point of the character 
        encoded = FME(ascii_val, e, n) # setting encoded to the FME algorithm
        cipher_text.append(encoded) # appending the string together of the ASCII encoded values of the cipher text
    return cipher_text

encoded = Encode(n, e, "What time is it?") #input here the text to be encoded
encoded

[868860133601557,
 13211641791567831,
 20436916271957116,
 8302426957980714,
 19942730250851363,
 8302426957980714,
 27102085920647667,
 5185778824741955,
 25824199538612172,
 19942730250851363,
 27102085920647667,
 19270259444554612,
 19942730250851363,
 27102085920647667,
 8302426957980714,
 21071392622714283]

In [10]:
def Decode(n, d, cipher_text):
    """
    Here, the cipher_text will be a list of integers.
    First, you will decrypt each of those integers using 
    n and d.
    Later, you will need to use the function that converts the integers to a string 
    defined in second toolset
    to recover the original message as a string. 
    
    """
    message = '' #setting up the message
    for letter in cipher_text: #creating for loop to decode each character
        decoded = FME(letter, d, n) #using the FME to decode using letter, d and n values
        ascii_val = chr(decoded) #finding the integer representing the unicode point of the character
        message += ascii_val #putting the message together
    return message

decoded = Decode(n, d, encoded) #the decoded message using n, d, and the encoded message

decoded


'What time is it?'

#### Now the fun begins:

In [11]:

if __name__ == '__main__':
    """"
    Implement a main function such that:
    
    1. Asks the user to Get Keys, Encode or Decode.
    2. If Getting keys, it will need to ask for p and q. 
    3. If Encoding, it will need the message and public keys.
    4. If Decoding it will need the coded message, and public and private keys.
    
    You may implement this differently if you want, add more functions, 
    include a list of useful primes etc... 
    Just implement in a way that makes sense to you.
    And explain why.
    
    
    USING A MAIN FUNCTION IS OPTIONAL if you feel you do not have enough experince. 
    You will need to organize your functions within the Jupyter Notebook 
    so we can follow what you are doing.
    
    """
    # (set some defaults)
    choice = "?"
    p = 13
    q = 17
    e = 739 #I used a prime number found from https://en.wikipedia.org/wiki/Prime_number
    n = 743 #Same here, used a prime from https://en.wikipedia.org/wiki/Prime_number
    d = 77
    print("1. Get Keys")
    print("2. Encode")
    print("3. Decode")
    print("0. Exit")
    while choice != "0": #while it's not zero to exit
        print('-----------------')
        print("p=" + str(p) + ", q=" + str(q) + ", e=" + str(e) + ", n=" + str(n) + ", d=" + str(d)) #setting up the menu options
        choice = input("Menu: ") #setting up the menu options
        if choice == "1":
            p = int(input("What value for p? "))
            q = int(input("What value for q? "))
            e, n = Find_Public_Key_e(p, q)
            d, n = Find_Private_Key_d(e, p, q)
        elif choice == "2":
            message = input("Input message: ")
            cipher_text = Encode(n, e, message)
            print(cipher_text)
        elif choice == "3":
            cipher_text = input("Input cipher text (including [square brackets]): ")
            cipher_text = cipher_text[1:-1]
            cipher_text = cipher_text.split(', ')
            cipher_text = [int(num) for num in cipher_text]
            message = Decode(n, d, cipher_text)
            print(message)
        elif choice == "0":
            print("You shall be missed.")
        else:
            print("You have failed.  Please try again.")
    
    print("Program Terminated")


1. Get Keys
2. Encode
3. Decode
0. Exit
-----------------
p=13, q=17, e=739, n=743, d=77


Menu:  0


You shall be missed.
Program Terminated


### 4. 

### Demonstrate how your RSA works below, by encoding and decoding a sample message here using your main above and sample inputs. Describe how this works and why you choose to put it together the way you did.


In [12]:
## "4" demo here in mix of text and code blocks

In [13]:
1. Get Keys
2. Encode
3. Decode
0. Exit
-----------------
p=13, q=17, e=5, n=221, d=77
Menu:  2
Input message:  Greetings, Earth creature.
[158, 173, 186, 186, 12, 209, 145, 103, 98, 57, 2, 205, 54, 173, 12, 117, 2, 216, 173, 186, 54, 12, 104, 173, 186, 37]
-----------------
p=13, q=17, e=5, n=221, d=77
Menu:  3
Input cipher text (including [square brackes]):  [158, 173, 186, 186, 12, 209, 145, 103, 98, 57, 2, 205, 54, 173, 12, 117, 2, 216, 173, 186, 54, 12, 104, 173, 186, 37]
Greetings, Earth creature.
-----------------
p=13, q=17, e=5, n=221, d=77
Menu:  0
You shall be missed.
Program Terminated

SyntaxError: invalid syntax (<ipython-input-13-45e3db4272e7>, line 1)

In [14]:
1. Get Keys
2. Encode
3. Decode
0. Exit
-----------------
p=13, q=17, e=5, n=221, d=77
Menu:  1
What value for p?  100003
What value for q?  100019
-----------------
p=100003, q=100019, e=5, n=10002200057, d=8001600029
Menu:  2
Input message:  Zzyzx is a weird name for a road.
[5904900000, 7022681518, 5933024487, 7022681518, 4878799886, 33554432, 2760615568, 109171761, 33554432, 8587340257, 33554432, 3859136485, 507900444, 2760615568, 9251945767, 10000000000, 33554432, 6102899943, 8587340257, 5384039492, 507900444, 33554432, 1038607975, 6848381494, 9251945767, 33554432, 8587340257, 33554432, 9251945767, 6848381494, 8587340257, 10000000000, 205962976]
-----------------
p=100003, q=100019, e=5, n=10002200057, d=8001600029
Menu:  3
Input cipher text (including [square brackes]):  [5904900000, 7022681518, 5933024487, 7022681518, 4878799886, 33554432, 2760615568, 109171761, 33554432, 8587340257, 33554432, 3859136485, 507900444, 2760615568, 9251945767, 10000000000, 33554432, 6102899943, 8587340257, 5384039492, 507900444, 33554432, 1038607975, 6848381494, 9251945767, 33554432, 8587340257, 33554432, 9251945767, 6848381494, 8587340257, 10000000000, 205962976]
Zzyzx is a weird name for a road.

SyntaxError: invalid syntax (<ipython-input-14-586a99fed19c>, line 1)

### 5.
**Next we will put it all together:**

* Find your public and private keys with  Find_Public_Key_e and Find_Private_Key_d

* Create digital messages with Encode.

* Decode with Decode.

* Publish your public and key and message to Piazza.

* Exchange private key with other classmates to Piazza.

* Test!


**Insert Narrative here in this block - One or two pages describing the results of exchanging keys and codes with classmates.**

* What was challenging? 
### Without a doubt the most challenging part was maintaining all of the variables, and how they interact, in my mind as the algorithm was pieced together. Also challenging was remembering to use the Extended Euclidean Algorithm to find d, and to use Fast Modular Exponentiation to encrypt and decrypt. When I was first able to decrypt a message, it was exhilirating and exciting, especially given the constant test & error until it finally worked.

* Who helped you with your project? 
### I got help from the websites listed below, office hour questions and comments from fellow classmates. I also used a tutor to help me debug my code and provide a check on my mathelatical thinking.

* Which resources were most helpful?
* https://www.geeksforgeeks.org/sieve-of-eratosthenes/
* https://www.khanacademy.org/computing/computer-science/cryptography/modarithmetic/a/the-euclidean-algorithm
* https://en.wikipedia.org/wiki/Modular_exponentiation
* https://en.wikipedia.org/wiki/Euclidean_algorithm
* https://en.wikipedia.org/wiki/List_of_prime_numbers#The_first_1000_prime_numbers
* https://primes.utm.edu/lists/small/10000.txt
* https://crypto.stackexchange.com/questions/19444/rsa-given-q-p-and-e I used this one after several hours of not finding a better way to find gcd


* Best Mistake?
### I made many mistakes but I think the best (or most frustrating) was to discover, after referring back to my notes over and over, was to figure out how to distinguish between "d" and "e".  I finally after several days was able to discover "e" was for "encrypt" and "d" is used for "decrypt".



### 6.
**RSA part 2:**

After you have tested your RSA package yourself, test it with classmates by publishing both the private and public keys on Piazza category for this (this will be a folder separate from Project) for a message. Then post 3 messages with just the public key for people to break - use both very small n ‘s (under 1000) for practice, and some with a challenge.

* To break others’ code you just need to factor n (public key).

* Write simple “brute force” method for finding for prime factors of n and use this to try to crack codes.

* To improve this factoring algorithm, consider THM 2 from page 258.


**Implementing a factoring algorithm:**

Implement the basic brute force factorization algorithm shown as pseudo code below to find prime factors of n.

Brute Force Factoring

 def factorize(n):
      # n is a number, return the smallest factor of n
      for i from 2 to n-1:
          if i divides n:
	      return i
      return FALSE


In [15]:
## "6"  Include your own code for brute force method of finding factors. (just rewrite above and cpmment the code)

def get_primes_to_sqrt(n):
    # we know that the prime we find must be less than or equal to the square root of n
    # [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    # [F  F  T  T  T  T  T  T  T  T   T   T   T] 
    cap = int((n+1)**0.5)+1 # set the cap to the square root of n+1 (plus an extra 1 for wiggle room)
    # primes variable will represent whether the number at that index is prime.  eg: primes[7] == True, primes[8] == False
    primes = [True]*cap # create that many trues (so if number is 10,000, cap will be 101, and 101*101 >= 10,000)
    primes[0] = False # 0 can't be prime
    primes[1] = False # 1 also can't be prime
    p = 1 # so starting from 1
    
    while p < cap: # as long as p is less than the cap
        p = p + 1 # add 1 to it
        while p < cap and primes[p] == False: # and as long as primes[p] has already been determiend as not prime
            p = p + 1 # add 1 until it hits a number that IS prime (since it's not divisible by any smaller numbers)
        for i in range(p*p, cap, p): # now starting from p, make every p'th number non-prime (not counting p)
            primes[i] = False 
            # 0f 1f 2T 3T 4f 5T 6f 7T 8f 9f 10f 11T 12f 13 14f 15f 16f 17 18f 19 20f 21f 22f 23..
            # note; I START AT p*p because EVERYTHING between 11 and 121 that's divisble by 11 would have to be divisible by a number < 11
            # eg: 77 looks like it might be prime, but it's divisble by 11, and ALSO by 7 (which is less than 11) so no need to check 77.
    prime_numbers = [] # prepare the prime number box
    for i in range(0, cap): # go through the entire collection of trues and falses
        if primes[i] == True: # and if a value is true
            prime_numbers.append(i) # that index (i) is prime
    return prime_numbers # return the numbers
    



### 7.
* Describe your attempts to "break" codes for which you do not have the private key. What happens as you try to factor large "n"s?
### The Jupyter tab slows down considerably as I attempt to break codes with very large n values.  However, with the smaller n values, it is not even noticeable as the program tries to run through all of the options.

* What happens as you try to factor very large numbers?
### When I try to factor very large numbers the Jupyter tab slows down, and even stops working entirely (at least from my point of view) and I need to hard stop the operation to get my computer back.






In [16]:
## Note that this only works when main is not running for some reason (I'm not sure why this is the case)

def get_p_q_from_n(n):
    possibilities = get_primes_to_sqrt(n) #setting up the possibilities from get primes to square root of n
    print("Prime count: " + str(len(possibilities)))
    for possibility in possibilities: #setting for loop 
        if n%possibility==0: #if n modulo possibility is zero
            p = possibility #we set p equal to possibility
            q = n//p #setting q equal to n divided by p
            return (p, q) #here we return p and q
    return None

def get_private_key_from_public_key(e, n):
    p, q = get_p_q_from_n(n) #setting p and q equal to get_p_q_from_n(n)
    m = (p-1)*(q-1) #setting m per the RSA given equation for m
    d = egcd(e, m) #setting d equal to egcd of e and m
    while d <= 0: #if d is negative making it positive
        d += m #d plus m until it's positive
    return d, n


e = 113
n = 28794290241716387
(d, n) = get_private_key_from_public_key(e, n)
(d, n)

Prime count: 9486701


(23697955405914833, 28794290241716387)

In [20]:
message = [2579706886152660, 1506880588871094, 25824199538612172, 12324506914914625, 3910629358746840, 19942730250851363, 12407119628852595, 20436916271957116, 22358653510094960, 22358653510094960, 18133059900110149, 19942730250851363, 18350208408879280, 12324506914914625, 27102085920647667, 28009467377098228, 19348565858364802, 19942730250851363, 8479118201845191, 28009467377098228, 27102085920647667, 19348565858364802, 8302426957980714, 19270259444554612, 19942730250851363, 8302426957980714, 13211641791567831, 25824199538612172, 19942730250851363, 8097851867855643, 20436916271957116, 3910629358746840, 18133059900110149]

Decode(n, d, message)



'Every Fall, Orion points the way,'

### 8.
How many codes did you break? Give some examples.


In [ ]:
### I had some success in breaking many of the messages with smaller n values. However, as the n 
### values got extremely large, my Jupyter Lab tab slowed significantly and i had to stop the 
### program as it was taking too long!

### Alana Bellucci 
### Public key (e, n) = (5, 3781)
### Encoded message = [2225, 1115, 1906, 1906, 2804, 1838, 2225, 1115, 945, 945, 3025, 864, 1867, 1867, 2339, 2043]
### Decoded d = 713
### Decoded message = Happy Halloween!


### Thomas Dolan
### Public key (e, n) = (7, 3599)
### Encoded message = [1567, 1918, 2591, 1951, 2634, 1297, 2591, 268, 1391, 3130, 2591, 1918, 1391, 268, 1701, 2634, 3322, 2591, 1701, 734, 2349, 2946, 2591, 1749, 2591, 371, 2349, 2619, 2349, 1701, 2591, 3130, 3386, 2591, 588, 1297, 371, 2634, 2946, 2113]
### Decoded d = 2983
### Decoded message: 'If you can factor this 4 digit n, kudos.'

### Professor Stade
### Public key: [113, 28794290241716387]
### Decoded d: 23697955405914833
### Encoded message = [2579706886152660, 1506880588871094, 25824199538612172, 12324506914914625, 3910629358746840, 19942730250851363, 12407119628852595, 20436916271957116, 22358653510094960, 22358653510094960, 18133059900110149, 19942730250851363, 18350208408879280, 12324506914914625, 27102085920647667, 28009467377098228, 19348565858364802, 19942730250851363, 8479118201845191, 28009467377098228, 27102085920647667, 19348565858364802, 8302426957980714, 19270259444554612, 19942730250851363, 8302426957980714, 13211641791567831, 25824199538612172, 19942730250851363, 8097851867855643, 20436916271957116, 3910629358746840, 18133059900110149]
### Decoded message: 'Every Fall, Orion points the way,'


### 9. 

Now implement at least **two** improvements to your factoring method. 

To improve the brute force method - a very naive and slow approach in many ways. We recommend considering improvements in the following order.

**Basic Improvement #1 :** Skip over certain numbers (pick at least one of these):

The brute force approach above tests even numbers as well as odd. If the number is odd, there is no point searching for an even factor.

We can do a better job tackling multiples of 3. If the number is not a multiple of 3, then no factor can be a multiple of 3 either.

How about multiples of 5,7,11,13,…?

**Basic Improvement #2: The brute force approach iterates all the way to n−1n−1. Can we stop earlier?**

To improve the brute force method - a very naive and slow approach in many ways. 

Hint: Explain with examples how using THM 3 page 258 could improve the brute force method of factoring by implementing (and/or other methods) to improve the brute force method.

Then add this technique to your factoring code to improve.


Include some narrative at the top of you code or in a text block about your choices.

See advanced options in the project description in Moodle.

 



## for 9 include code for improvements to factoring algorithm.  Please comment


### Improvmeent 1. I implemented Sieve of Eratosthenes - we are able to skip through the p+p (in the official version) by simply multiplying by p.  
### My refinement starts at p*p, and counts by p. Because nothing between p and p*p could be divisible by p without also 
### being divisble by a number smaller than p.
### Improvement 2. I am capping it out at the square root of the target number - which is also a huge improvement
### Any number that is greater than the square root of our target and can divide our target must be paired with a number that is smaller than the square root of our target.


def get_primes_to_sqrt(n):
    # we know that the prime we find must be less than or equal to the square root of n
    # [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    # [F  F  T  T  T  T  T  T  T  T   T   T   T] 
    cap = int((n+1)**0.5)+1 # set the cap to the square root of n+1 (plus an extra 1 for wiggle room)
    # primes variable will represent whether the number at that index is prime.  eg: primes[7] == True, primes[8] == False
    primes = [True]*cap # create that many trues (so if number is 10,000, cap will be 101, and 101*101 >= 10,000)
    primes[0] = False # 0 can't be prime
    primes[1] = False # 1 also can't be prime
    p = 1 # so starting from 1
    
    while p < cap: # as long as p is less than the cap
        p = p + 1 # add 1 to it
        while p < cap and primes[p] == False: # and as long as primes[p] has already been determiend as not prime
            p = p + 1 # add 1 until we hit a number that IS prime (since it's not divisible by any smaller numbers)
        for i in range(p*p, cap, p): # now starting from p, make every p'th number non-prime (not counting p)
            primes[i] = False 
            # 0f 1f 2T 3T 4f 5T 6f 7T 8f 9f 10f 11T 12f 13 14f 15f 16f 17 18f 19 20f 21f 22f 23..
            # note; WE START AT p*p because EVERYTHING between 11 and 121 that's divisble by 11 would have to be divisible by a number < 11
            # eg: 77 looks like it might be prime, but it's divisble by 11, and ALSO by 7 (which is less than 11) so no need to check 77.
    prime_numbers = [] # prepare the prime number box
    for i in range(0, cap): # go through the entire collection of trues and falses
        if primes[i] == True: # and if a value is true
            prime_numbers.append(i) # that index (i) is prime
    return prime_numbers # return the numbers

### 10.
*Conclusion: Return to the questions and comments you asked about in the RSA in homework about Number Theory. Answer your questions and/or explain what you have learned.
*Q1. What makes two primes good choices for determining n? Are there good or bad primes? 
### Yes. There are "good" and "bad" primes for choosing n. As it turns out, the smaller the n the easier it might be for someone to crack your code.  Generally speaking, the larger the n, the more difficult you make it for someone to brute force discover your hidden key.

*Q2. If the message is very long, wouldn't someone be able to decipher your message by looking for a
pattern?
### Yes. In fact, short messages with a given context can be deciphered. This is even mentioned at the end of the week by the professor given an example using [843, 2371, 2371, 2310, 1262, 4723, 58, 4115, 1560, 4431] and the context of closing out week.

*Q3. If RSA relies so heavily on the difficulty of factoring prime numbers, if ever someone figures out an easier way to do it, would the world economy be able to quickly pivot to a different way of encrypting messages?
### This is not clear. Since I now know RSA is the backbone of the entire internet, and is the most copied software in the world, it seems unlikely that we would be able to quickly pivot to a new encrypting algorithm, nor is it clear if one even exists!
 



## Grading ##
    
Grading - all code must be commented.

**Max grade 60 pts** - Successful implementation of the 1 - 4 , and 10. Clear and comprehensive write up. Do not neglect introduction.

**Max grade 80 pts** - add #5, and  must include testing code and exchanging keys at least 3 times. Clear and comprehensive write-up.

#You must be exchanging public and private keys (thru #5 ) a few days early (see Moodle for date) before the project is due to  get credit for the final 20 point of the project. (#6/7/8/9)

**Max grade 100 pts** - Do all of #1-#10.  **Must be exchanging codes with keys by Sat. noon (weekend before the project is due) to do (#6/7/8/9)** 
Clear and comprehensive write-up.


Do NOT copy code from the internet (or anywhere)
Do not ask questions on stackoverflow or similar etc…
You are expected to have read the honestly notes in the syllabus.

Credit sources you used for additional understanding.
Credit classmates that helped you.

